Model 1 Test Data Analysis
==========================
Table of Contents
-----------------
1. [Data Loading](#Data-Loading)
2. [Latitudinal Analysis](#Latitudinal-Analysis)
3. [Longitudinal Analysis](#Longitudinal-Analysis)
4. [Time Series Analysis](#Time-Series-Analysis)
5. [Conclusion](#Conclusion)

In [33]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [34]:
# Set the model number for the model that was trained
modelNum = 1

# Data Loading

In [35]:
df = pd.read_csv(f'../Models/Model_1/Model_{modelNum}_TestData.csv')
df.head()

,Lat,Lon,Alt,Precip (mm),Temp (°C),Year,JulianDaySin,O18 (‰) Actual,H2 (‰) Actual,O18 (‰) Predicted,H2 (‰) Predicted
0,37.302680,-2.901980,1773.0,0.000000,554.302100,2016.0,-0.271958,-12.13,-82.30,-9.175214,-63.488710
1,28.220000,-177.370000,13.0,136.000000,26.300000,1974.0,-0.213521,-1.81,-5.20,-1.088409,-1.546110
2,47.120000,9.490000,450.0,4.274954,555.598486,2013.0,-0.974100,-13.70,-102.15,-9.598681,-68.047630
3,46.398641,6.233804,436.0,0.000000,553.571753,2020.0,-0.711657,-7.92,-56.84,-10.350645,-73.997280
4,-7.315278,72.428056,1.0,578.000000,573.957068,1987.0,0.951057,-3.04,-15.50,-4.571165,-25.942806


## Preparing the dataframe for analysis
Currently the dataframe looks good but in order to preform some analysis we will need to clean it up just a bit. We will start by converting the date to a datetime object and then by changing column names to not include units of measurement.

In [36]:
# Convert Year and Julian Sine Day to a single date
df['JulianDay'] = np.ceil((np.arcsin(df['JulianDaySin']) / np.pi + 0.5) * 365).astype(int)
df['Year'] = df['Year'].astype(int)

# Combine the Year and Julian Day into a single date
df['Date'] = pd.to_datetime(df['Year'].astype(str) + '/' + df['JulianDay'].astype(str), format='%Y/%j')

In [37]:
# Remove the Year and Julian Day columns
df = df.drop(['Year', 'JulianDay', 'JulianDaySin'], axis=1)

In [38]:
# Cycle through the column names and change the names to be more usable in code
import re
oldColumns = df.columns
codeCols = list(map(lambda x: re.sub(r'\(.*?\)', '', x), oldColumns))

# Cycle through the column names and change ones that end in Actual or Predicted to be more usable in code (remove the word Actual or Predicted)
codeCols = list(map(lambda x: re.sub(r'Actual', 'A', x), codeCols))
codeCols = list(map(lambda x: re.sub(r'Predicted', 'P', x), codeCols))
codeCols = list(map(lambda x: x.strip(), codeCols))

# Create a dictionary to map the old column names to the new column names
colDict = dict(zip(oldColumns, codeCols))

# Rename the columns
df = df.rename(columns=colDict)

# Set the Date column as the index
df = df.set_index('Date')

# Sort the columns by date
df = df.sort_index()

df

,Lat,Lon,Alt,Precip,Temp,O18 A,H2 A,O18 P,H2 P
Date,,,,,,,,,
1960-04-03,51.930000,-10.250000,9.0,60.0,14.800000,-3.700,-23.00,-4.531493,-29.158308
1961-01-27,22.316667,114.166667,66.0,31.0,25.800000,-5.500,-49.00,-4.193994,-24.466526
1961-01-27,-40.350000,-9.880000,54.0,290.0,9.400000,-2.700,-14.90,-3.876350,-20.788540
1961-01-27,31.183333,29.950000,7.0,34.0,21.800000,-5.700,-26.70,-2.897123,-13.187976
1961-01-27,19.880000,102.130000,305.0,76.0,23.100000,-11.400,-79.90,-7.478314,-53.350815
...,...,...,...,...,...,...,...,...,...
2022-12-24,47.816667,13.717778,1618.0,0.6,557.845496,-8.647,-86.07,-9.961754,-70.736496
2022-12-29,47.816667,13.717778,1618.0,0.7,554.424078,-6.318,-38.49,-9.958141,-70.685330
2022-12-31,47.816667,13.717778,1618.0,13.4,555.377814,-15.763,-119.83,-10.219426,-72.771080
